## Measuring VNC length

The VNC length is measured using a centerline estimation.
The steps are as follows:

1. The image is binarized, so we can apply a distance transform.
The distance transform is very succeptible to holes and concavities, so we also remove holes from the binary image and apply a binary opening to smooth edges.
2. We apply a 'chessboard' distance transform (minimum difference between two points along any coordinate dimension) to the binary image, which naturally results in peaks over the centerline of the binary image.
Given the embryo morphology, we expect several center points along the anterior-posterior axis and eventually another set of points towards the brain lobes.
3. To separate these points, we fit a RANSAC model that, given the majority of the points are along the AP axis, fits a line along the VNC centerline.
4. From this line intersection to the binary image, we are able to measure the VNC length.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
from tifffile import imread

from pasnascope import centerline, find_hatching, utils, vnc_length

experiment_name = '20240503'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', experiment_name)

In [ ]:
img_dir = project_dir.joinpath('embs')
imgs = sorted(img_dir.glob('*ch2.tif'), key=utils.sort_by_emb_name)
img_path = imgs[2]
print(img_path.stem)

img = imread(img_path, key=1020)
centerline.view_centerline_dist(img)

In [ ]:
def get_length_centerline(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    return vnc_length.measure_VNC_centerline(img)*pixel_width

interval = 20
img_dir = project_dir.joinpath('embs')
annotated_dir = project_dir.joinpath('annotated')
embs = [f.stem for f in annotated_dir.iterdir()][:2]
print(embs)

lengths = []
ann_lengths = []

for emb in embs:
    img_path = img_dir / f"{emb}.tif"
    hp = find_hatching.find_hatching_point(img_path)
    hp -= hp % interval

    vnc_len = get_length_centerline(img_path, hp, interval=interval)
    lengths.append(vnc_len)

    csv_path = annotated_dir / f"{emb}.csv"
    csv_end = hp//interval
    manual = vnc_length.get_length_from_csv(csv_path, csv_end)
    ann_lengths.append(manual)

fig, ax = plt.subplots(len(embs))
ax = ax.ravel()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('Centerline estimation')
for i, (l, al) in enumerate(zip(lengths, ann_lengths)):
    x = list(range(0, len(l)*interval, interval))
    ax[i].plot(x, l, color='r', label='calculated')
    ax[i].plot(x, al, color='g', label='annotated')
ax[0].legend()

plt.tight_layout()

In [ ]:
def get_length_centerline(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    return vnc_length.measure_VNC_centerline(img)*pixel_width

interval = 20
img_dir = project_dir.joinpath('embs')
imgs = sorted(img_dir.glob('*ch2.tif'), key=utils.sort_by_emb_name)
i = 1
img_path = imgs[i]
print(img_path.stem)

hp = find_hatching.find_hatching_point(img_path)
print(hp)

vnc_len = get_length_centerline(img_path, hp, interval=interval)

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle(f'VNC length estimation for {img_path.stem}')

x = list(range(0, len(vnc_len)*interval, interval))
ax.plot(x, vnc_len)

plt.tight_layout()

Visualize how much variability in the centerline estimation we have, since RANSAC in inherently non-deterministic.

In [ ]:
def get_length_centerline(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    return vnc_length.measure_VNC_centerline(img)*pixel_width

interval = 20
img_dir = project_dir.joinpath('embs')
imgs = sorted(img_dir.glob('*ch2.tif'), key=utils.sort_by_emb_name)
i = 0
num_replicates = 10

img_path = imgs[i]
print(img_path.stem)

replicates = []

for i in range(num_replicates):
    hp = find_hatching.find_hatching_point(img_path)

    vnc_len = get_length_centerline(img_path, hp, interval=interval)
    replicates.append(vnc_len)

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle(f'VNC length estimation for {img_path.stem} (n=10)')

x = list(range(0, len(replicates[0])*interval, interval))
for i,l in enumerate(replicates):
    ax.plot(x, l)

plt.tight_layout()